In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import subprocess
import os
import pandas as np
import numpy as np
import pathlib
import re
import glob
import pandas as pd

module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from workbench.utilities import general, config_utils, temporal, io
from workbench.manage import manage
from workbench.host import host
from workbench.workflows import workflows

In [87]:
par_dir = "/Users/jmccarty/Desktop/test_project"
project_name = 'test1'
base_epw = "/Users/jmccarty/Desktop/CHE_ZH_Zurich.AP.066700_TMYx.2007-2021.epw"
device_name = 'mini-monocrystalline'
module_orientation = 'portrait'

config_file = manage.initiate_project(par_dir, project_name, base_epw, device_name, module_orientation)
my_proj = manage.Project(config_file)
my_proj.project_setup()

Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/shared/tmy/base.epw
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/0_workbench_geometry_template.3dm
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/0_workbench_geometry_template.gh
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/1_workbench_panelizer_template.gh
The project is initialized. We have created a base host object named 'B1000'.
You will need to either move or create the geometry and panelizer files into the appropriate directories.
The geometry files should follow the convention defined in the output of the template grasshopper and rhino files.
The panelizer files are those that have been created using the grasshopper utility. The Panelizer is
not ready

# Workflow Description

The radiance workflow in Workbench employs the 2-Phase DDS described in [1]. This enhanced 2-Phase method of Radiance simulates direct and diffuse contributions on a sensor grid. The workflow is summarised below and was based on the explanation given in [2]:  

Part 1 Total Irradiance:  
1. Generate Octree with [oconv]
2. Run [rfluxmtx]
   1. requires the grid file
   2. skyglow generated dynamically based on configuration resolution
   3. dependent on existence of Accerlerad 
   4. requires the rad parameters from a configruation file
3. Generate sky patches with [gendaymtx] and [epw2wea] 
   1. a filter is placed on the output of [epw2wea] from the configuration HOY range
4. Calculate the results using [dctimestep] and [rmtxop]
   1. total.ill

Part 2 Direct Irradiance:  
1. Generate black Octree with [oconv]
2. run [rfluxmtx]
   1. requires the grid file
   2. use [reviously] established skyglow file
   3. same rad parameters except the -ab is set to 1
3. Generate sky patches with [gendaymtx]
   1. use previously created wea file
4. Calculate the results using [dctimestep] and [rmtxop]
   1. direct.ill
   
Part 3 Sun Coefficients:  
1. Generate a primitive sun object
   1. void light solar 0 0 3 1e6 1e6 1e6
2. Create solar discs using [cnt] and [rcalc]
   1. input is the reinsrc.cal for MF:6
3. Create another octree [oconv] 
   1. black
4. Caclulate direct illumiance coefficients using [rcontrib]
   1. requires the grid file
   2. dependent on existence of accelerad
   3. requires radiance parameters inputs from config file
5. Create a new sun matrix with [gendaymtx]
   1. related tot he subdivisions number used in 3.2 (MF:6)
6. Calculate the results using [dctimestep] and [rmtxop]
   1. sun.ill

Part 4 Calculate Results:  
1. Calculate the diffuse contribution
   1. total.ill - direct.ill
2. Convert .ill to feather format
   1. The .ill files require significant space due to the highly discretized grid.
   2. We employ feather thorugh pyarrow to reduce the write time and size



[1] D. Bourgeois, C. Reinhart, and G. Ward, ‘Standard daylight coefficient model for dynamic daylighting simulations’, Building Research and Information, vol. 36, Jan. 2008, doi: 10.1080/09613210701446325.  
[2] S. Subramaniam, ‘Daylighting Simulations with Radiance using Matrix-based Methods’, LBNL Energy Technologies Area, Berkeley, Oct. 2017.

# Functions

In [4]:
def create_black_objects(input_f, output_f):
    cmd = f"echo !xform -m black {input_f} > {output_f}"
    subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)

def build_cmd_oconv(radiance_project_dir, radiance_surface_key, step):
    cmd = ['oconv']

    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    oct_dir = os.path.join(rad_surface_dir,"outputs","octree")
    utils.directory_creator(oct_dir)

    object_material_file = os.path.join(rad_surface_dir, "model","scene","envelope.mat")
    object_file = os.path.join(rad_surface_dir, "model","scene","envelope.rad")
    black_object_material_file = os.path.join(rad_surface_dir, "model","scene","envelope.blk")
    black_object_file = os.path.join(rad_surface_dir, "model","scene","envelope_black.rad")
    if os.path.exists(black_object_file):
        pass
    else:
        create_black_objects(object_file, black_object_file)
    
    glazing_material_file = os.path.join(rad_surface_dir, "model","aperture","aperture.mat")
    glazing_file = os.path.join(rad_surface_dir, "model","aperture","aperture.rad")
    black_glazing_material_file = os.path.join(rad_surface_dir, "model","aperture","aperture.blk")
    black_glazing_file = os.path.join(rad_surface_dir, "model","aperture","aperture_black.rad")

    if os.path.exists(black_glazing_file):
        pass
    else:
        create_black_objects(glazing_file, black_glazing_file)
        
    sun_file = os.path.join(rad_surface_dir, "model","scene","suns.rad")
    
    if step=="total":
        oct_name = "total.oct"
        output_file = os.path.join(oct_dir,oct_name)
        cmd+=[object_material_file, object_file, glazing_material_file, glazing_file]#, ">", output_file]
        
    elif step=="direct":
        oct_name = "direct.oct"
        output_file = os.path.join(oct_dir,oct_name)
        cmd+=[black_object_material_file, black_object_file, glazing_material_file, glazing_file]#, ">", output_file]
        
    elif step=="sun":
        oct_name = "sun.oct"
        output_file = os.path.join(oct_dir,oct_name)
        if os.path.exists(sun_file):
            cmd+=["-f", black_object_material_file, black_object_file, sun_file,  glazing_material_file, glazing_file]#, ">", output_file]
        else:
            print("The file 'suns.rad' is missing. This can be generated from a primiary solar material and then 'rcalc' with a reinhart sky.")
            return FileNotFoundError
        
    else:
        print("Arg 'step' must be specified as 'total', 'direct', or 'sun'")
        return TypeError
    
    return cmd, output_file


def create_skyglow(skyglow_file, resolution, dst):
    """_summary_

    Args:
        skyglow_file (str): the tempalte skyglow file
        resolution (int): An integer from 1 to 6 that determines the number of sky subdivisions for the skyglow. 1 is a Tregenza sky.
        dst (str): where to save the new skyglow file
    """
    with open(skyglow_file, "r") as fp:
        skyglow_lines = fp.readlines()
        
    new_lines = []
    for l in skyglow_lines:
        if "h=r" in l:
            n_l = re.sub("\d+(?=\D*$)", resolution, l)
            new_lines.append(n_l)
        else:
            new_lines.append(l)

    with open(dst, "w") as fp:
        fp.writelines(new_lines)
    

def build_cmd_rfluxmtx(radiance_project_dir, radiance_surface_key, skyglow_template, sky_resolution, step, n_workers=None, rad_params=None):
    cmd = ["rfluxmtx", "-I+"]
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    output_dir = os.path.join(rad_surface_dir, "outputs", "matrices")
    utils.directory_creator(output_dir)
    
    skyglow_file = os.path.join(rad_surface_dir, "model","scene","skyglow.rad")
    if os.path.exists(skyglow_file):
        pass
    else:
        create_skyglow(skyglow_template, str(sky_resolution), skyglow_file)
        
    oct_dir = os.path.join(rad_surface_dir,"outputs","octree")
    if step == 'total':
        octree_file = os.path.join(oct_dir, "total.oct")
        output_file = os.path.join(output_dir, 'total_illum.mtx')
    elif step =='direct':
        octree_file = os.path.join(oct_dir, "direct.oct")
        output_file = os.path.join(output_dir, 'direct_illum.mtx')
    else:
        print("Arg 'step' must be specified as 'total' or 'direct'")
        return TypeError
        
    grid_file = glob.glob(os.path.join(rad_surface_dir, "model","grid","*.pts"))[0]
    line_count = int(grid_file.split("_")[-1].split("s")[0])
    # lines in grid file
    # line_count = utils.count_lines(grid_file)
    cmd+=["-y", f"{int(line_count)}"]
    
    # number of workers
    if n_workers is None:
        n_workers = os.cpu_count() - 1
    cmd+=["-n", f"{int(n_workers)}"]
    
    # rad_params
    if rad_params is None:
        rad_params =  "-lw 0.0001 -ab 5 -ad 10000"
    cmd+=rad_params.split(" ")
    
    # skyglow
    cmd+=["-", f"{skyglow_file}"]
    
    # sender and octree
    cmd+=["-i", f"{octree_file}", "<", f"{grid_file}"]#, ">", f"{output_file}"]
    
    return cmd, output_file, grid_file

def build_cmd_epw2wea(radiance_project_dir, radiance_surface_key, input_epw):
    input_epw = pathlib.Path(input_epw)
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    wea_name = input_epw.name.replace(".epw",".wea")
    
    output_wea = os.path.join(rad_surface_dir, 'model', wea_name)
    
    cmd = ['epw2wea']
    
    cmd+=[input_epw]
    cmd+=[output_wea]
    return cmd, output_wea

def build_cmd_gendaymtx(radiance_project_dir, radiance_surface_key, wea_file, step):
    """_summary_
    example: gendaymtx -m 1 assets/NYC.wea > skyVectors/NYC.smx
    sun-coe example: gendaymtx -5 0.533 -d -m 6 assets/NYC.wea > skyVectors/NYCsunM6.smx
    Args:
        radiance_project_dir (_type_): _description_
        radiance_surface_key (_type_): _description_
        wea_file (_type_): _description_
        step (_type_): _description_

    Returns:
        _type_: _description_
    """
    cmd = ['gendaymtx']
    
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    output_dir = os.path.join(rad_surface_dir, "outputs", "matrices")
    
    if step == 'total':
        output_file = os.path.join(output_dir, 'sky_total.smx')
        cmd+=['-m', '1']
    elif step =='direct':
        output_file = os.path.join(output_dir, 'sky_direct.smx')
        cmd+=['-m', '1', '-d']
    elif step == 'sun':
        output_file = os.path.join(output_dir, 'sky_sun.smx')
        cmd+=['-5', '0.533', '-d', '-m', '6']
    else:
        print("Arg 'step' must be specified as 'total', 'direct', or 'sun'")
        return TypeError
    
    cmd+=[wea_file]
    
    return cmd, output_file
    


def build_cmd_dctimestep(radiance_project_dir, radiance_surface_key, step):
    """_summary_
    example: dctimestep matrices/cds/cdsDDS.mtx skyVectors/NYCsunM6.smx
    Args:
        radiance_project_dir (_type_): _description_
        radiance_surface_key (_type_): _description_
    """

    cmd = ['dctimestep']
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    matrices_dir = os.path.join(rad_surface_dir, "outputs", "matrices")
    
    if step == 'total':
        input_matrix = os.path.join(matrices_dir, 'total_illum.mtx')
        input_sky = os.path.join(matrices_dir, 'sky_total.smx')
    elif step =='direct':
        input_matrix = os.path.join(matrices_dir, 'direct_illum.mtx')
        input_sky = os.path.join(matrices_dir, 'sky_direct.smx')
    elif step == 'sun':
        input_matrix = os.path.join(matrices_dir, 'sun_illum.mtx')
        input_sky = os.path.join(matrices_dir, 'sky_sun.smx')
    else:
        print("Arg 'step' must be specified as 'total', 'direct', or 'sun'")
        return TypeError
    
    cmd+=[input_matrix, input_sky]
    
    return cmd
    
def build_cmd_rmtxop(radiance_project_dir, radiance_surface_key, step):
    """_summary_
    The three numbers specified after -c convert the irradiance values 
    to illuminance [lux] by scaling and combining the result according to the photopic efficiency function.
    The conversion from lux to W/m2 is applied at the end of the workflow when all result .ill are combined.
    
    example: rmtxop -fa -t -c 47.4 119.9 11.6 - > results/dcDDS/dc/annualR.ill
    Args:
        radiance_project_dir (_type_): _description_
        radiance_surface_key (_type_): _description_
    """

    cmd = ['rmtxop']
    cmd+=["-fa", "-t", "-c", "47.4", "119.9", "11.6", "-"]
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    output_dir = os.path.join(rad_surface_dir, "outputs", "results")
    utils.directory_creator(output_dir)
    
    if step == 'total':
        output_file = os.path.join(output_dir, 'result_total.ill')
    elif step =='direct':
        output_file = os.path.join(output_dir, 'result_direct.ill')
    elif step == 'sun':
        output_file = os.path.join(output_dir, 'result_sun.ill')
    else:
        print("Arg 'step' must be specified as 'total', 'direct', or 'sun'")
        return TypeError

    return cmd, output_file

def create_primitive_sun(radiance_project_dir, radiance_surface_key):
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    scene_dir = os.path.join(rad_surface_dir, "model", "scene")
    output_file = os.path.join(scene_dir, "suns.rad")
    write_line = "void light solar 0 0 3 1e6 1e6 1e6\n"

    with open(output_file, 'w') as file:
        file.write(write_line)
    
    return output_file
    
def build_cmd_cnt():
    cmd = ["cnt"]
    
    cmd+=['5165']
    
    return cmd

def build_cmd_rcalc(cal = 'reinsrc.cal'):
    cmd = ['rcalc']

    cmd+=['-e', 'MF:6', '-f', cal, '-e', 'Rbin=recno', '-o', r"'solar source sun 0 0 4 ${Dx} ${Dy} ${Dz} 0.533'"]
    
    return cmd


def build_cmd_rcontrib(radiance_project_dir, radiance_surface_key, cal = "reinhart.cal", n_workers=None, rad_params=None):
    cmd = ["rcontrib", "-I+", "-ab", "1"]
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    output_dir = os.path.join(rad_surface_dir, "outputs", "matrices")
    utils.directory_creator(output_dir)
    output_file = os.path.join(output_dir, 'sun_illum.mtx')
    
    sun_oct = os.path.join(rad_surface_dir, "outputs", "octree", "sun.oct")
    # rcontrib -I+ -ab 1 -y 100 -n 16 -ad 256 -lw 1.0e-3 -dc 1 -dt 0 -dj 0 -faf -e MF:6 -f reinhart.cal -b rbin -bn Nrbins -m solar octrees/sunCoefficientsDDS.oct < points.txt > matrices/cds/cdsDDS.mtx
    
    
    grid_file = glob.glob(os.path.join(rad_surface_dir, "model","grid","*.pts"))[0]
    line_count = int(grid_file.split("_")[-1].split("s")[0])
    # lines in grid file
    # line_count = utils.count_lines(grid_file)
    cmd+=["-y", f"{int(line_count)}"]
    
    # number of workers
    if n_workers is None:
        n_workers = os.cpu_count() - 1
    cmd+=["-n", f"{int(n_workers)}"]
    
    # rad_params
    if rad_params is None:
        rad_params = "-ad 256 -lw 1.0e-3 -dc 1 -dt 0 -dj 0"
    cmd+=rad_params.split(" ")
    
    cmd+=["-faf", "-e", "MF:6", "-f", cal, "-b", "rbin", "-bn", "Nrbins", "-m", "solar", sun_oct]
    
    
    return cmd, output_file, grid_file

'/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench/workbench/template_files/skyglow.rad'

In [5]:
# INputs

project_dir = "/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/radiance"
rad_surface = "1000_0"
step = 'sun'
rflux_rad_params = "-lw 0.0001 -ab 5 -ad 10000" #config
sky_resolution = 1 #config

analysis_period = '0-8760' # config
year = 2030 # confifg

rcontrib_rad_params = "-ad 256 -lw 1.0e-3 -dc 1 -dt 0 -dj 0" # config

In [260]:
## Run oconv

cmd_oconv, output = build_cmd_oconv(project_dir, rad_surface, step)
with open(output, 'w') as fp:
    proc = subprocess.run(cmd_oconv, check=True, input=None, stdout=fp)


In [261]:
## Run rfluxmtx

cmd_rfluxmtx, output_file, input_file = build_cmd_rfluxmtx(project_dir, rad_surface, my_proj.skyglow_template, sky_resolution, step,
                                                           n_workers=7, rad_params=rflux_rad_params)


with open(input_file, "rb") as fp:
    proc = subprocess.run(cmd_rfluxmtx, check=True, stdin=fp,
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=5)

with open(output_file, "wb") as fp:
    fp.write(proc.stdout)


Arg 'step' must be specified as 'total' or 'direct'


TypeError: cannot unpack non-iterable type object

In [262]:
## Run epw2wea and filter wea
# build command
cmd_epw2wea, output_wea = build_cmd_epw2wea(project_dir, rad_surface, my_proj.TMY_FILE)

# run command
proc = subprocess.run(cmd_epw2wea, check=True, input=None, stdout=subprocess.PIPE)

# filter wea
time_utils.filter_wea(output_wea, year, analysis_period)

In [263]:
## Run gendaymtx
cmd_gendaymtx, output_file = build_cmd_gendaymtx(project_dir, rad_surface, output_wea, step)

with open(output_file, 'w') as file:
    subprocess.run(cmd_gendaymtx, stdout=file, check=True)

gendaymtx: warning - unusually bright at 7.5 on 2-8
gendaymtx: warning - unusually bright at 7.5 on 2-9
gendaymtx: warning - unusually bright at 7.5 on 2-10
gendaymtx: warning - unusually bright at 7.5 on 2-11
gendaymtx: warning - unusually bright at 7.5 on 2-12
gendaymtx: warning - unusually bright at 7.5 on 2-13
gendaymtx: warning - unusually bright at 7.5 on 2-14
gendaymtx: warning - unusually bright at 7.5 on 2-15
gendaymtx: warning - unusually bright at 7.5 on 2-16
gendaymtx: warning - unusually bright at 7.5 on 2-18
gendaymtx: warning - unusually bright at 7.5 on 2-19
gendaymtx: warning - unusually bright at 6.5 on 3-13
gendaymtx: warning - unusually bright at 6.5 on 3-14
gendaymtx: warning - unusually bright at 6.5 on 3-15
gendaymtx: warning - unusually bright at 6.5 on 3-16
gendaymtx: warning - unusually bright at 6.5 on 3-17
gendaymtx: warning - unusually bright at 6.5 on 3-18
gendaymtx: warning - unusually bright at 6.5 on 3-19
gendaymtx: warning - unusually bright at 6.5 on 

In [271]:
## Run dctimestep and rmtxop

# First command: dctimestep matrices/dc/illum.mtx skyVectors/NYC.smx
cmd_dctimestep = build_cmd_dctimestep(project_dir, rad_surface, step)
proc_dctimestep = subprocess.Popen(cmd_dctimestep, stdout=subprocess.PIPE)

# Second command: rmtxop -fa -t -c 47.4 119.9 11.6 -
cmd_rmtxop, output_file = build_cmd_rmtxop(project_dir, rad_surface, step)
proc_rmtxop = subprocess.Popen(cmd_rmtxop, stdin=proc_dctimestep.stdout, stdout=subprocess.PIPE)

with open(output_file, 'wb') as file:
    file.write(proc_rmtxop.communicate()[0])

In [265]:
## Create solar discs sun file

# Create a primitive sun
prim_sun_file = create_primitive_sun(project_dir, rad_surface)

# build comand for count
cmd_cnt = build_cmd_cnt()

# build rcalc command
cmd_rcalc = build_cmd_rcalc()

# merge and point to file
full_cmd = " ".join(cmd_cnt)+ " | " + " ".join(cmd_rcalc) + f" >> {prim_sun_file}"

# run
subprocess.run(full_cmd, shell=True)

CompletedProcess(args="cnt 5165 | rcalc -e MF:6 -f reinsrc.cal -e Rbin=recno -o 'solar source sun 0 0 4 ${Dx} ${Dy} ${Dz} 0.533' >> /Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/radiance/surface_1000_0/model/scene/suns.rad", returncode=0)

In [269]:
## Run rcontrib

cmd_rcontrib, output_file, input_file = build_cmd_rcontrib(project_dir, rad_surface, n_workers=7, rad_params=rcontrib_rad_params)


with open(input_file, "rb") as fp:
    proc = subprocess.run(cmd_rcontrib, check=True, stdin=fp,
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=5)

with open(output_file, "wb") as fp:
    fp.write(proc.stdout)


In [26]:
## Write final result Files

def ill_to_df(radiance_project_dir, radiance_surface_key):
    lux_to_wattm2 = 0.0079
    rad_surface_dir = os.path.join(radiance_project_dir, f"surface_{radiance_surface_key}")
    output_dir = os.path.join(rad_surface_dir, "outputs", "results")

    filepath_total = os.path.join(output_dir, 'result_total.ill')
    filepath_direct = os.path.join(output_dir, 'result_direct.ill')
    filepath_sun = os.path.join(output_dir, 'result_sun.ill')
    
    df_total = utils.read_ill(filepath_total)
    df_direct = utils.read_ill(filepath_direct)
    df_sun = utils.read_ill(filepath_sun)
    
    indirect_illuminance = df_total - df_direct
    
    direct = df_sun * lux_to_wattm2
    direct = direct.astype("float").round(2)
    indirect = indirect_illuminance * lux_to_wattm2
    indirect = indirect.astype("float").round(2)
    indirect = pd.DataFrame(np.where(indirect < 0, 0, indirect))
    return direct, indirect



In [17]:
direct, indirect = ill_to_df(project_dir, rad_surface)

# Testing

In [89]:
rad_surface = "1000_5"
my_proj.edit_cfg_file('analysis', 'active_surface', rad_surface)
er = radiance.run_2phase_dds(my_proj)


Running Radiance workflow for 2-Phase DDs with 4 workers
Initializing the opening weather file.
Starting Part 1 (total).
 - oconv
 - rfluxmtx
 - gendaymtx
 - dctimestep | rmtxop
Starting Part 2 (direct).
 - oconv
 - rfluxmtx
 - gendaymtx
 - dctimestep | rmtxop
Starting Part 3 (sun).
 - create_primitive_sun
 - rcalc
 - oconv
 - rcontrib
 - gendaymtx
 - dctimestep | rmtxop


In [90]:
radiance.save_irradiance_results(my_proj)

Saving Irradiance results
Direct sensor data saved in compressed format, time=0.0-seconds.
Diffuse sensor data saved in compressed format, time=0.0-seconds.


In [105]:
par_dir = "/Users/jmccarty/Desktop/test_project"
project_name = 'test1'
base_epw = "/Users/jmccarty/Desktop/CHE_ZH_Zurich.AP.066700_TMYx.2007-2021.epw"
device_id = 'D0013'

config_file = manage.initiate_project(par_dir, project_name, base_epw, device_id)
my_proj = manage.Project(config_file)
my_proj.project_setup()

Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/shared/tmy/base.epw
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/0_workbench_geometry_template.3dm
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/0_workbench_geometry_template.gh
Destination file already exists, copy aborted./n/Users/jmccarty/Desktop/test_project/test1/inputs/hosts/B1000/base/geometry/1_workbench_panelizer_template.gh
The project is initialized. We have created a base host object named 'B1000'.
You will need to either move or create the geometry and panelizer files into the appropriate directories.
The geometry files should follow the convention defined in the output of the template grasshopper and rhino files.
The panelizer files are those that have been created using the grasshopper utility. The Panelizer is
not ready

In [99]:
my_host = host.Host(my_proj)

In [90]:
type(my_host.project.irradiance_store_radiance)

bool

In [34]:
my_host.project.edit_cfg_file("irradiance","n_workers",7)

In [49]:
workflows.run_irradiance(my_host)

Starting Radiance workflow for surface 1000_1.
 - Running 2-Phase DDS with 7 workers
 - Current surface is 1000_1
 - Initializing the opening weather file.
 - Starting Part 1 (total).
     - oconv
     - rfluxmtx
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 2 (direct).
     - oconv
     - rfluxmtx
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 3 (sun).
     - create_primitive_sun
     - rcalc
     - oconv
     - rcontrib
     - gendaymtx
     - dctimestep | rmtxop
Completed in 42.26 seconds.
Converting .ill files to feather for surface 1000_1.
 - Saving Irradiance results
    - Direct sensor data saved in compressed format, time=0.0-seconds.
    - Diffuse sensor data saved in compressed format, time=0.0-seconds.
Completed in 0.59 seconds.
-----------------------
Starting Radiance workflow for surface 1000_8.
 - Running 2-Phase DDS with 7 workers
 - Current surface is 1000_8
 - Initializing the opening weather file.
 - Starting Part 1 (total).
     - oconv
